## Import libraries

In [ ]:
from PIL import Image
from keras.layers import *
from keras.applications import *
from keras import *
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_hub as hub

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14197080870678910413
]


## Global parameters

In [ ]:
IMG_HEIGHT = 224
IMG_WIDTH = 224

STEPS_EPOCHS = 930
EPOCHS = 1

STEPS_PREDICT = 1861

## Creation of the dataset

In [ ]:
def load_image(img, training):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) / 127.5 - 1
    return tf.image.resize(img, (IMG_HEIGHT, IMG_WIDTH))


def load_triplets(triplet, training):
    ids = tf.strings.split(triplet)
    paths = ['food/' + ids[0] + '.jpg','food/' + ids[1] + '.jpg','food/' + ids[2] + '.jpg']
    anchor = load_image(tf.io.read_file(paths[0]), training)
    truthy = load_image(tf.io.read_file(paths[1]), training)
    falsy = load_image(tf.io.read_file(paths[2]), training)
    return tf.stack([anchor, truthy, falsy], axis=0),1
    
    
def create_dataset(dataset_filename):
    dataset = tf.data.TextLineDataset(dataset_filename)
    dataset = dataset.map(lambda triplet: load_triplets(triplet,True))
    return dataset

## Custom loss & accuracy function

In [ ]:
def triplet_loss(_,predictions): # we use a triplet loss in this case like for facenet
    anchor, correct, wrong = predictions[...,0],predictions[...,1],predictions[...,2]
    distance_correct = tf.reduce_sum(tf.square(anchor - correct),axis=1)
    distance_false = tf.reduce_sum(tf.square(anchor - wrong),axis=1)
    return tf.reduce_mean(tf.math.softplus(distance_correct - distance_false))
   

def accuracy(_,predictions):
    anchor, correct, wrong = predictions[...,0],predictions[...,1],predictions[...,2]
    distance_correct = tf.reduce_sum(tf.square(anchor - correct),axis=1)
    distance_false = tf.reduce_sum(tf.square(anchor - wrong),axis=1)
    return tf.reduce_mean(tf.cast(tf.greater_equal(distance_false,distance_correct), tf.float32))

## Creation of the siamese nn

In [ ]:
def create_model():
    input_layer = Input(shape=(3,IMG_HEIGHT, IMG_WIDTH, 3))
    
    anchor = input_layer[:,0,...]
    correct = input_layer[:,1,...]
    wrong = input_layer[:,2,...]
    
    model_url = "https://tfhub.dev/google/bit/m-r50x1/1"
    encoder = hub.KerasLayer(model_url, trainable=False)
    
    decoder = Sequential([
        Dropout(0.3),
        Dense(256),
        Activation('relu'),
        Dropout(0.3),
        Dense(128),
        Activation('sigmoid')       
    ])
    
    output_layer = tf.stack([decoder(encoder(anchor)),decoder(encoder(correct)),decoder(encoder(wrong))],axis=-1)
    
    model = Model(inputs=input_layer,outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss = triplet_loss,
                   metrics=[accuracy])
    
    return model

model = create_model()

## Predictior

In [ ]:
def getTestData(dataset_filename):
    dataset = tf.data.TextLineDataset(dataset_filename)
    dataset = dataset.map(lambda triplet: load_triplets(triplet,False))
    dataset = dataset.batch(32).prefetch(10)
    return dataset


def predictor(model):
    anchor, correct, wrong = model.output[...,0],model.output[...,1],model.output[...,2]
    distance_correct = tf.reduce_sum(tf.square(anchor - correct),1)
    distance_false = tf.reduce_sum(tf.square(anchor - wrong),1)
    predictions = tf.cast(tf.greater_equal(distance_false,distance_correct), tf.int8)
    return tf.keras.Model(inputs=model.inputs, outputs=predictions)

## Train the model

In [ ]:
train_dataset = create_dataset('train_triplets.txt')
train_dataset = train_dataset.shuffle(1024, reshuffle_each_iteration=True).repeat().batch(64)

history = model.fit(
    train_dataset,
    steps_per_epoch= STEPS_EPOCHS,
    epochs=EPOCHS, # One epochs is enough
)

## Output model

In [ ]:
inference_model = predictor(model)

input_data = getTestData('test_triplets.txt')

output = inference_model.predict(input_data,steps=STEPS_PREDICT)

In [ ]:
np.savetxt('predictions.txt',output,fmt='%i')